In [156]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sshtunnel import SSHTunnelForwarder
import psycopg2 as psy
import math

In [157]:
def get_conn(SSH_required,key_path):   #for getting a connection as a result

    db='datawarehouse'
    DB_HOST='datawarehouse.cdgpvetprks3.ap-south-1.rds.amazonaws.com'
    conn = []
    if SSH_required == 'Yes':
        SSH_HOST='ec2-15-206-161-154.ap-south-1.compute.amazonaws.com'
        #LOCALHOST="0.0.0.0"
        ssh_tunnel= SSHTunnelForwarder(
                (SSH_HOST, 22),
                ssh_username="ec2-user",
                ssh_private_key= key_path,
                ssh_private_key_password= "",
                remote_bind_address=(DB_HOST, 5432),
                local_bind_address=('127.0.0.1', 0)
        )
        print('Tunnel Started')
        ssh_tunnel.start()
        conn = psy.connect(
            host=ssh_tunnel.local_bind_host,
            port=ssh_tunnel.local_bind_port,
            user='postgres',
            password= "Simply1234",
            database='postgres')
        print('Connection Made')
        return conn
    else:
        conn = psy.connect(
            host = DB_HOST,
            port = 5432,
            user = 'postgres',
            password= "Simply1234",
            database='postgres')
        print('Connection Made')
        return conn

In [158]:
def get_df_from_sql(SSH_required, query,key_path):   #for getting a datafarame as a result

    db='datawarehouse'
    DB_HOST='datawarehouse.cdgpvetprks3.ap-south-1.rds.amazonaws.com'
    conn = None
    if SSH_required == 'Yes':
        SSH_HOST='ec2-15-206-161-154.ap-south-1.compute.amazonaws.com'
        #LOCALHOST="0.0.0.0"
        ssh_tunnel= SSHTunnelForwarder(
                (SSH_HOST, 22),
                ssh_username="ec2-user",
                ssh_private_key= key_path,
                ssh_private_key_password= "",
                remote_bind_address=(DB_HOST, 5432),
                local_bind_address=('127.0.0.1', 0)
        )
        # ssh_tunnel._server_list[0].block_on_close = False
        ssh_tunnel.start()
        conn = psy.connect(
            host=ssh_tunnel.local_bind_host,
            port=ssh_tunnel.local_bind_port,
            user='postgres',
            password= "Simply1234",
            database='postgres')
        df_results = pd.read_sql(query, conn)
        conn.close()
        ssh_tunnel.stop()
        return df_results
    else:
        conn = psy.connect(
            host = DB_HOST,
            port = 5432,
            user = 'postgres',
            password= "Simply1234",
            database='postgres')
        df_results = pd.read_sql(query, conn)
        conn.close()
        return df_results

In [159]:
# Usage with the actual path to the private key
SSH_required = 'Yes'
key_path = '/Users/rajatsansaniwal/Documents/tunnel-ssh .cer'
query_path = 'pickup_query.sql'
with open(query_path,'r') as file:
    query = file.read()

# Establish a connection
conn = get_conn(SSH_required, key_path)

# Retrieve data into a DataFrame
df = get_df_from_sql(SSH_required, query, key_path)

# Now you can perform further operations with the DataFrame 'df'
print(df)

Tunnel Started
Connection Made
                 key1  warehouse_id pickup_vehicle          pickuptime  \
0      100-1514-Delhi          1514        Vishu.. 2024-07-07 21:14:28   
1      100-1514-Delhi          1514        Vishu.. 2024-07-07 21:15:16   
2      100-1514-Delhi          1514        Vishu.. 2024-07-07 21:15:20   
3      100-1514-Delhi          1514        Vishu.. 2024-07-07 21:16:13   
4      100-1514-Delhi          1514        Vishu.. 2024-07-07 21:16:41   
...               ...           ...            ...                 ...   
17934   87-327-Mumbai           327         mujeeb 2024-07-07 18:32:34   
17935   87-327-Mumbai           327         mujeeb 2024-07-07 18:32:37   
17936   87-327-Mumbai           327         mujeeb 2024-07-07 18:32:44   
17937   87-327-Mumbai           327         mujeeb 2024-07-07 18:32:50   
17938   87-327-Mumbai           327         mujeeb 2024-07-07 18:32:57   

                awb  
0      GS2038142560  
1      GS2101548470  
2      GS13896

In [163]:
df['pickuptime'] = pd.to_datetime(df['pickuptime'])

results = []

for (key1, warehouse, pickup_vehicle), group in df.groupby(['key1', 'warehouse_id', 'pickup_vehicle']):
    group = group.reset_index(drop=True)

    pickup_start_time = group.loc[0, 'pickuptime']

    for i in range(1, len(group)):
        current_time = group.loc[i, 'pickuptime']
        previous_time = group.loc[i - 1, 'pickuptime']
        
        # Check the difference between current and previous pickuptime
        if (current_time - previous_time).total_seconds() > 7200:  # More than two hours
            # End the current pickup slot and start a new one
            results.append({
                'key1': key1,
                'warehouse_id': warehouse,
                'pickup_vehicle': pickup_vehicle,
                'pickup_start_time': pickup_start_time,
                'pickup_end_time': previous_time
            })
            pickup_start_time = current_time  # Update the start time for the new slot
    
    # Append the last slot for the warehouse
    results.append({
        'key1': key1,
        'warehouse_id': warehouse,
        'pickup_vehicle': pickup_vehicle,
        'pickup_start_time': pickup_start_time,
        'pickup_end_time': group.loc[len(group) - 1, 'pickuptime']
    })

# Create DataFrame from results list
results_df = pd.DataFrame(results)

# Calculate slot_duration in minutes
results_df['slot_duration'] = ((results_df['pickup_end_time'] - results_df['pickup_start_time']).dt.total_seconds() / 60).apply(math.ceil)

# Sort DataFrame by warehouse_id and pickup_start_time (if not already sorted)
results_df.sort_values(by=['key1', 'warehouse_id', 'pickup_start_time'], inplace=True)

# Calculate gaps between pickup slots in minutes
results_df['gap_to_next'] = results_df.groupby(['key1', 'warehouse_id', 'pickup_vehicle'])['pickup_start_time'].shift(-1) - results_df['pickup_end_time']
results_df['gap_to_next'] = (results_df['gap_to_next'].dt.total_seconds() / 60)
results_df['gap_to_next'].fillna(0, inplace=True)
results_df['gap_to_next'] = (results_df['gap_to_next'].astype(int) + 1).replace(1, np.nan)

results_df.sort_values(by=['key1', 'warehouse_id', 'pickup_vehicle'], inplace=True)

print(results_df)

                   key1  warehouse_id pickup_vehicle   pickup_start_time  \
0        100-1514-Delhi          1514        Vishu.. 2024-07-07 21:14:28   
1        100-1514-Delhi          1514     Yellapa Ji 2024-07-07 11:19:47   
2     100-329-Bangalore           329        Vishu.. 2024-07-07 21:14:22   
3     100-329-Bangalore           329     Yellapa Ji 2024-07-07 11:19:35   
4    101-1400-Hyderabad          1400   Chakali Raju 2024-07-07 18:55:06   
..                  ...           ...            ...                 ...   
163  449-2043-Bangalore          2043      Manjunath 2024-07-07 12:09:23   
164  449-2043-Bangalore          2043        Vishu.. 2024-07-07 20:31:05   
165      452-2140-Delhi          2140    Sohan singh 2024-07-07 12:03:30   
166       87-1025-Delhi          1025      Karamveer 2024-07-07 09:51:18   
167       87-327-Mumbai           327         mujeeb 2024-07-07 18:32:25   

        pickup_end_time  slot_duration  gap_to_next  
0   2024-07-07 21:24:13          

In [161]:
loads = []

for index, row in results_df.iterrows():
    key1 = row['key1']
    warehouse_id = row['warehouse_id']
    pickup_vehicle = row['pickup_vehicle']
    slot_start_time = row['pickup_start_time']
    slot_end_time = row['pickup_end_time']
    
    load = 0  # Initialize load count for this slot

    filtered_df = df[(df['key1'] == key1) &
                     (df['warehouse_id'] == warehouse_id) &
                     (df['pickup_vehicle'] == pickup_vehicle) &
                     (df['pickuptime'] >= slot_start_time) &
                     (df['pickuptime'] <= slot_end_time)]

    # Count the number of AWBs within the slot time range and add to load
    load += len(filtered_df)
    
    # Append the total load count for this slot to the loads list
    loads.append(load)

# Assign the 'actual_load' column to results_df using the loads list
results_df['actual_load'] = loads

# Print or display the updated DataFrame with the 'actual_load' column
print(results_df)

# Save DataFrame to Excel
excel_file = 'pickup_slots.xlsx'
results_df.to_excel(excel_file, index=False)

print(f"DataFrame saved to {excel_file}")


                   key1  warehouse_id pickup_vehicle   pickup_start_time  \
0        100-1514-Delhi          1514        Vishu.. 2024-07-07 21:14:28   
1        100-1514-Delhi          1514     Yellapa Ji 2024-07-07 11:19:47   
2     100-329-Bangalore           329        Vishu.. 2024-07-07 21:14:22   
3     100-329-Bangalore           329     Yellapa Ji 2024-07-07 11:19:35   
4    101-1400-Hyderabad          1400   Chakali Raju 2024-07-07 18:55:06   
..                  ...           ...            ...                 ...   
163  449-2043-Bangalore          2043      Manjunath 2024-07-07 12:09:23   
164  449-2043-Bangalore          2043        Vishu.. 2024-07-07 20:31:05   
165      452-2140-Delhi          2140    Sohan singh 2024-07-07 12:03:30   
166       87-1025-Delhi          1025      Karamveer 2024-07-07 09:51:18   
167       87-327-Mumbai           327         mujeeb 2024-07-07 18:32:25   

        pickup_end_time  slot_duration  gap_to_next  actual_load  
0   2024-07-07 21:24